# **Tutorial** - Topic Modeling with BERTopic
(last updated 01-09-2022)

In this tutorial we will be exploring how to use BERTopic to create topics from the well-known 20Newsgroups dataset. The most frequent use-cases and methods are discussed together with important parameters to keep a look out for. 


## BERTopic
BERTopic is a topic modeling technique that leverages 🤗 transformers and a custom class-based TF-IDF to create dense clusters allowing for easily interpretable topics whilst keeping important words in the topic descriptions. 

<br>

<img src="https://raw.githubusercontent.com/MaartenGr/BERTopic/master/images/logo.png" width="40%">

# Enabling the GPU

First, you'll need to enable GPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select GPU from the Hardware Accelerator drop-down

[Reference](https://colab.research.google.com/notebooks/gpu.ipynb)

# **Installing BERTopic**

We start by installing BERTopic from PyPi:

In [1]:
%%capture
!pip install bertopic

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/email_bodies.zip


## Restart the Notebook
After installing BERTopic, some packages that were already loaded were updated and in order to correctly use them, we should now restart the notebook.

From the Menu:

Runtime → Restart Runtime

# Data
For this example, we use the popular 20 Newsgroups dataset which contains roughly 18000 newsgroups posts

In [4]:
import os
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords

print()


docs=list()
FOLDER='email_bodies'
stopwords_=stopwords.words('english')

for i in os.listdir(FOLDER):
  with open(f'{FOLDER}/{i}', 'r') as file:
    doc=file.read()
    doc=doc.split()
    for j in stopwords_:
      while j in doc:
        doc.remove(j)
    doc=' '.join(doc)
    docs.append(doc)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# **Topic Modeling**

In this example, we will go through the main components of BERTopic and the steps necessary to create a strong topic model. 




## Training

We start by instantiating BERTopic. We set language to `english` since our documents are in the English language. If you would like to use a multi-lingual model, please use `language="multilingual"` instead. 

We will also calculate the topic probabilities. However, this can slow down BERTopic significantly at large amounts of data (>100_000 documents). It is advised to turn this off if you want to speed up the model. 


In [5]:
from bertopic import BERTopic

topic_model = BERTopic(language="english", calculate_probabilities=False, verbose=True)
topics = topic_model.fit_transform(docs)

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/612 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

Batches:   0%|          | 0/3748 [00:00<?, ?it/s]

2022-11-30 15:41:28,488 - BERTopic - Transformed documents to Embeddings
2022-11-30 15:45:22,081 - BERTopic - Reduced dimensionality
2022-11-30 15:45:41,133 - BERTopic - Clustered reduced embeddings


**NOTE**: Use `language="multilingual"` to select a model that support 50+ languages.

## Extracting Topics
After fitting our model, we can start by looking at the results. Typically, we look at the most frequent topics first as they best represent the collection of documents. 

In [6]:
freq = topic_model.get_topic_info(); freq

,Topic,Count,Name
0,-1,60845,-1_cc_subject_forwarded_re
1,0,657,0_meeting_conference_suzanne_kay
2,1,589,1_recipient_intended_estoppel_enronmessagingad...
3,2,493,2_intended_notify_message_privileged
4,3,446,3_sent_sent09thursday_thu_sent09tuesday
...,...,...,...
1762,1761,10,1761_missed_master_weather_masters
1763,1762,10,1762_mrhajulie_gorte_dash_gomez
1764,1763,10,1763_forms_olegalattyfmsswap_subdirectories_fo...
1765,1764,10,1764_looks_good_great_add


-1 refers to all outliers and should typically be ignored. Next, let's take a look at a frequent topic that were generated:

In [9]:
topic_model.get_topic(1000)

[('paula', 0.09480269108373715),
 ('rebecca', 0.05125340463985238),
 ('dual', 0.04698932567767356),
 ('rieker', 0.03999122114968947),
 ('variety', 0.03376073020957069),
 ('bachelors', 0.03171930163892485),
 ('endeavors', 0.029020082805045023),
 ('midoctober', 0.02875841125116752),
 ('capacities', 0.028568953017285695),
 ('served', 0.026694351836753845)]

**NOTE**: BERTopic is stocastich which mmeans that the topics might differ across runs. This is mostly due to the stocastisch nature of UMAP.

In [ ]:
### Attributes

## Attributes

There are a number of attributes that you can access after having trained your BERTopic model:


| Attribute | Description |
|------------------------|---------------------------------------------------------------------------------------------|
| topics_               | The topics that are generated for each document after training or updating the topic model. |
| probabilities_ | The probabilities that are generated for each document if HDBSCAN is used. |
| topic_sizes_           | The size of each topic                                                                      |
| topic_mapper_          | A class for tracking topics and their mappings anytime they are merged/reduced.             |
| topic_representations_ | The top *n* terms per topic and their respective c-TF-IDF values.                             |
| c_tf_idf_              | The topic-term matrix as calculated through c-TF-IDF.                                       |
| topic_labels_          | The default labels for each topic.                                                          |
| custom_labels_         | Custom labels for each topic as generated through `.set_topic_labels`.                                                               |
| topic_embeddings_      | The embeddings for each topic if `embedding_model` was used.                                                              |
| representative_docs_   | The representative documents for each topic if HDBSCAN is used.                                                |

For example, to access the predicted topics for the first 10 documents, we simply run the following:

In [ ]:
topic_model.topics_[:10]

[178, -1, 18, -1, 1036, 168, 380, -1, 208, 568]

# **Visualization**
There are several visualization options available in BERTopic, namely the visualization of topics, probabilities and topics over time. Topic modeling is, to a certain extent, quite subjective. Visualizations help understand the topics that were created. 

## Visualize Topics
After having trained our `BERTopic` model, we can iteratively go through perhaps a hundred topic to get a good 
understanding of the topics that were extract. However, that takes quite some time and lacks a global representation. 
Instead, we can visualize the topics that were generated in a way very similar to 
[LDAvis](https://github.com/cpsievert/LDAvis):

In [ ]:
topic_model.visualize_topics()

In [ ]:
!unzip drive/MyDrive/my_model.zip
!cp drive/MyDrive/my_model my_model

Archive:  drive/MyDrive/my_model.zip
  inflating: my_model                
cp: cannot stat 'drive/MyDrive/my_model': No such file or directory


## Visualize Topic Probabilities

The variable `probabilities` that is returned from `transform()` or `fit_transform()` can 
be used to understand how confident BERTopic is that certain topics can be found in a document. 

To visualize the distributions, we simply call:

In [10]:
topic_model.visualize_distribution(probs[150], min_probability=0.015)

NameError: ignored

## Visualize Topic Hierarchy

The topics that were created can be hierarchically reduced. In order to understand the potential hierarchical structure of the topics, we can use scipy.cluster.hierarchy to create clusters and visualize how they relate to one another. This might help selecting an appropriate nr_topics when reducing the number of topics that you have created.

In [12]:
topic_model.visualize_hierarchy(top_n_topics=150)

## Visualize Terms

We can visualize the selected terms for a few topics by creating bar charts out of the c-TF-IDF scores for each topic representation. Insights can be gained from the relative c-TF-IDF scores between and within topics. Moreover, you can easily compare topic representations to each other.

In [14]:
topic_model.visualize_barchart(top_n_topics=8)

## Visualize Topic Similarity
Having generated topic embeddings, through both c-TF-IDF and embeddings, we can create a similarity matrix by simply applying cosine similarities through those topic embeddings. The result will be a matrix indicating how similar certain topics are to each other.

In [15]:
topic_model.visualize_heatmap(n_clusters=10, width=1000, height=1000)

## Visualize Term Score Decline
Topics are represented by a number of words starting with the best representative word. Each word is represented by a c-TF-IDF score. The higher the score, the more representative a word to the topic is. Since the topic words are sorted by their c-TF-IDF score, the scores slowly decline with each word that is added. At some point adding words to the topic representation only marginally increases the total c-TF-IDF score and would not be beneficial for its representation.

To visualize this effect, we can plot the c-TF-IDF scores for each topic by the term rank of each word. In other words, the position of the words (term rank), where the words with the highest c-TF-IDF score will have a rank of 1, will be put on the x-axis. Whereas the y-axis will be populated by the c-TF-IDF scores. The result is a visualization that shows you the decline of c-TF-IDF score when adding words to the topic representation. It allows you, using the elbow method, the select the best number of words in a topic.


In [ ]:
topic_model.visualize_term_rank()

# **Topic Representation**
After having created the topic model, you might not be satisfied with some of the parameters you have chosen. Fortunately, BERTopic allows you to update the topics after they have been created. 

This allows for fine-tuning the model to your specifications and wishes. 

## Update Topics
When you have trained a model and viewed the topics and the words that represent them,
you might not be satisfied with the representation. Perhaps you forgot to remove
stopwords or you want to try out a different `n_gram_range`. We can use the function `update_topics` to update 
the topic representation with new parameters for `c-TF-IDF`: 


In [ ]:
topic_model.update_topics(docs, n_gram_range=(1, 2))

In [ ]:
topic_model.get_topic(0)   # We select topic that we viewed before

NameError: ignored

## Topic Reduction
We can also reduce the number of topics after having trained a BERTopic model. The advantage of doing so, 
is that you can decide the number of topics after knowing how many are actually created. It is difficult to 
predict before training your model how many topics that are in your documents and how many will be extracted. 
Instead, we can decide afterwards how many topics seems realistic:





In [ ]:
topic_model.reduce_topics(docs, nr_topics=60)

2021-10-17 06:05:02,666 - BERTopic - Reduced number of topics from 220 to 61


In [ ]:
# Access the newly updated topics with:
print(my_model.topics_)

[178, -1, 18, -1, 1036, 168, 380, -1, 208, 568, 67, -1, -1, -1, 55, -1, 383, 740, -1, -1, -1, 999, -1, 49, 192, -1, -1, -1, 401, -1, 103, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 837, -1, -1, 267, 540, 842, 212, -1, 78, 88, -1, -1, 374, -1, 0, -1, -1, -1, 21, -1, -1, 726, -1, -1, -1, -1, 97, 578, 360, -1, -1, -1, -1, -1, 1266, 295, 1223, 11, -1, -1, 120, 229, -1, -1, 200, -1, -1, -1, -1, 104, -1, 103, 247, 1203, 71, 854, -1, -1, 56, 431, 497, -1, -1, -1, 237, -1, 149, 25, -1, 544, 71, 63, -1, 551, 102, -1, 680, -1, 5, 1265, 1027, -1, -1, 41, 107, -1, 931, -1, -1, -1, -1, -1, 22, 34, -1, 549, 112, -1, -1, -1, 710, 430, 396, -1, -1, -1, 100, 371, -1, 859, 142, -1, -1, -1, -1, 187, 222, 106, -1, 97, -1, -1, -1, 371, 273, -1, -1, 548, -1, -1, 33, 201, -1, -1, 346, -1, 58, 1006, -1, -1, 197, 142, 244, 24, -1, 156, -1, 624, -1, -1, 185, 17, 127, -1, -1, 63, -1, -1, -1, -1, 508, -1, -1, -1, -1, 431, -1, -1, -1, 650, 810, -1, 85, 96, -1, -1, -1, 8, -1, -1, -1, 889, 381, 238, -1, -1, -1,

# **Search Topics**
After having trained our model, we can use `find_topics` to search for topics that are similar 
to an input search_term. Here, we are going to be searching for topics that closely relate the 
search term "vehicle". Then, we extract the most similar topic and check the results: 

In [24]:
similar_topics, similarity = topic_model.find_topics("limited time", top_n=5); similar_topics

[279, 482, 726, 137, 1164]

In [25]:
for i in similar_topics:
  print(topic_model.get_topic(i))

[('restrictions', 0.061370027280310865), ('area', 0.04661083299932859), ('mli', 0.030089330676540526), ('columbia', 0.021591654696745666), ('paybacks', 0.01713215221275681), ('nonfirm', 0.015492741929053266), ('pal', 0.014453263308051186), ('withdrawals', 0.013854945727305968), ('market', 0.013749694134218081), ('nominations', 0.013362466237124017)]
[('unavailable', 0.17268913197522692), ('mailbox', 0.10241856042341257), ('x31411', 0.08809199477698874), ('owa', 0.06969514720463565), ('blackberry', 0.06928886528510418), ('resides', 0.06814955826723268), ('temporarily', 0.06399249913956351), ('outlook', 0.0594579479431901), ('delayed', 0.049015910103672594), ('server', 0.04832614617582217)]
[('crawler', 0.3144916733317183), ('dayahead', 0.29370582445426446), ('lifetime', 0.21959513928306543), ('exceeded', 0.20202070636706781), ('intervention', 0.17441327745667587), ('manual', 0.15070006849653494), ('failed', 0.10470456841579935), ('download', 0.0953253271811416), ('log', 0.08731795559733

# **Model serialization**
The model and its internal settings can easily be saved. Note that the documents and embeddings will not be saved. However, UMAP and HDBSCAN will be saved. 

In [16]:
# Save model
topic_model.save("my_model2")	

/usr/local/lib/python3.7/dist-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [17]:
!zip -r my_model2.zip my_model2

  adding: my_model2 (deflated 24%)


In [18]:
!cp my_model2.zip /content/drive/MyDrive/my_model2.zip

In [19]:
# Load model
!pip install bertopic
from bertopic import BERTopic

my_model = BERTopic.load("my_model")	

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3021, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.7/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2815, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/base_command.py", line 180, in _main
    status = self.run(options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/cli/req_command.py", line 199, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py", line 385, in run
    conflicts = self._determine_conflicts(to_install)
 

FileNotFoundError: ignored

# **Embedding Models**
The parameter `embedding_model` takes in a string pointing to a sentence-transformers model, a SentenceTransformer, or a Flair DocumentEmbedding model.

## Sentence-Transformers
You can select any model from sentence-transformers here and pass it through BERTopic with embedding_model:



In [ ]:
topic_model = BERTopic(embedding_model="xlm-r-bert-base-nli-stsb-mean-tokens")

Or select a SentenceTransformer model with your own parameters:


In [ ]:
from sentence_transformers import SentenceTransformer

sentence_model = SentenceTransformer("distilbert-base-nli-mean-tokens", device="cpu")
topic_model = BERTopic(embedding_model=sentence_model, verbose=True)

Click [here](https://www.sbert.net/docs/pretrained_models.html) for a list of supported sentence transformers models.  
